In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from google.colab import files
files.upload()

Saving skin cancer1.zip to skin cancer1.zip


In [1]:
!unzip skin\ cancer1.zip

Archive:  skin cancer1.zip
  inflating: Skin cancer dataset/test/benign/1.jpg  
  inflating: Skin cancer dataset/test/benign/10.jpg  
  inflating: Skin cancer dataset/test/benign/1003.jpg  
  inflating: Skin cancer dataset/test/benign/1006.jpg  
  inflating: Skin cancer dataset/test/benign/1013.jpg  
  inflating: Skin cancer dataset/test/benign/1018.jpg  
  inflating: Skin cancer dataset/test/benign/1019.jpg  
  inflating: Skin cancer dataset/test/benign/1023.jpg  
  inflating: Skin cancer dataset/test/benign/1025.jpg  
  inflating: Skin cancer dataset/test/benign/1029.jpg  
  inflating: Skin cancer dataset/test/benign/1033.jpg  
  inflating: Skin cancer dataset/test/benign/1034.jpg  
  inflating: Skin cancer dataset/test/benign/1042.jpg  
  inflating: Skin cancer dataset/test/benign/1044.jpg  
  inflating: Skin cancer dataset/test/benign/105.jpg  
  inflating: Skin cancer dataset/test/benign/1055.jpg  
  inflating: Skin cancer dataset/test/benign/1061.jpg  
  inflating: Skin cancer da

In [8]:
import os

In [9]:
for dirname, _, filenames in os.walk('/content/Skin cancer dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/Skin cancer dataset/train/malignant/1432.jpg
/content/Skin cancer dataset/train/malignant/772.jpg
/content/Skin cancer dataset/train/malignant/385.jpg
/content/Skin cancer dataset/train/malignant/594.jpg
/content/Skin cancer dataset/train/malignant/1188.jpg
/content/Skin cancer dataset/train/malignant/726.jpg
/content/Skin cancer dataset/train/malignant/616.jpg
/content/Skin cancer dataset/train/malignant/1071.jpg
/content/Skin cancer dataset/train/malignant/116.jpg
/content/Skin cancer dataset/train/malignant/1454.jpg
/content/Skin cancer dataset/train/malignant/1500.jpg
/content/Skin cancer dataset/train/malignant/909.jpg
/content/Skin cancer dataset/train/malignant/481.jpg
/content/Skin cancer dataset/train/malignant/917.jpg
/content/Skin cancer dataset/train/malignant/802.jpg
/content/Skin cancer dataset/train/malignant/1423.jpg
/content/Skin cancer dataset/train/malignant/777.jpg
/content/Skin cancer dataset/train/malignant/19.jpg
/content/Skin cancer dataset/train/malign

In [10]:
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [11]:
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

In [12]:
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()